Table: Activity <br>
+--------------+---------+<br>
| Column Name | Type |<br>
+--------------+---------+<br>
| player_id | int |<br>
| device_id | int |<br>
| event_date | date |<br>
| games_played | int |<br>
+--------------+---------+<br>
- (player_id, event_date) is the primary key of this table.
- This table shows the activity of players of some game.
- Each row is a record of a player who logged in and played a number of games
- (possibly 0) before logging out on some day using some device.

##### Write a SQL query that reports the device that is first logged in for each player.
The query result format is in the following example:<br>
Activity table:<br>
+-----------+-----------+------------+--------------+<br>
| player_id | device_id | event_date | games_played |<br>
+-----------+-----------+------------+--------------+<br>
| 1 | 2 | 2016-03-01 | 5 |<br>
| 1 | 2 | 2016-05-02 | 6 |<br>
| 2 | 3 | 2017-06-25 | 1 |<br>
| 3 | 1 | 2016-03-02 | 0 |<br>
| 3 | 4 | 2018-07-03 | 5 |<br>
+-----------+-----------+------------+--------------+<br>
Result table:<br>
+-----------+-----------+<br>
| player_id | device_id |<br>
+-----------+-----------+<br>
| 1 | 2 |<br>
| 2 | 3 |<br>
| 3 | 1 |<br>
+-----------+-----------+<br>

##### PySpark Solution

In [0]:
from pyspark.sql.types import DateType
schema = "player_id int, device_id int, event_date string, games_played int"
data = [(1,	2, "2016-03-01", 5),
        (1,	2, "2016-05-02", 6),
        (2,	3, "2017-06-25", 1),
        (3,	1, "2016-03-02", 0),
        (3,	4, "2018-07-03", 5)]

df = spark.createDataFrame(data = data, schema = schema)

df = df.withColumn("event_date", df["event_date"].cast(DateType()))
df.display()

player_id,device_id,event_date,games_played
1,2,2016-03-01,5
1,2,2016-05-02,6
2,3,2017-06-25,1
3,1,2016-03-02,0
3,4,2018-07-03,5


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

windowSpec = Window.partitionBy("player_id").orderBy("event_date")

df2 = (df
       .withColumn("rank", rank().over(windowSpec))
       .select("player_id", "device_id")
       .filter("rank = 1")
       )

df2.show()

+---------+---------+
|player_id|device_id|
+---------+---------+
|        1|        2|
|        2|        3|
|        3|        1|
+---------+---------+

